# nbdev

## What is nbdev?

nbdev is a notebook-driven development platform. Simply write notebooks with lightweight markup and get high-quality documentation, tests, continuous integration, and packaging

## What will we learn in this notebook?

`nbdev` sounds pretty cool, but there's also a lot going on there. Maybe you haven't even heard of continuous integration before. In this notebook we will go over how to use modular nbdev to create a python module. This is a much less complex introduction than the nbdev end-to-end walkthrough, because we won't create a whole package that can be uploaded to PyPi or Anaconda. We will simply use notebooks to create clean python modules that we can use to build our dashboards and web apps. 


## Export code to modules with nb_export
- `#| export` directive to signal that a cell is to be exported
- `#| default_exp` directive to signal the module to export to
- `nbdev.export.nb_export` with the name of the notebook and export director

### End Goal

The end goal of this tutorial notebook is to use some of the cells in this notebook to create a python module that we can import to other notebooks. Let's say we want to create a helper function that prints all the traits and trait values of a widget.

In [ ]:
import ipywidgets as widgets

object_has_traits = widgets.IntSlider(description='Pick a Number')
object_has_traits

Okay, we want to make a function that prints out the traits belonging to this widget. We can use the `traits()` function to do so.

In [ ]:
trait_names = object_has_traits.trait_names()
trait_names

This looks interesting, but there's a lot of stuff there, especially with underscores, that we aren't intereted in. Let's get rid of those.

In [ ]:
trait_names = [name for name in trait_names if not name.startswith('_')]
trait_names

There are a few others in there we might to filter out before printing. 


In [ ]:
exclude = ['comm', 'log', 'keys']
trait_names = [name for name in trait_names if name not in exclude]
trait_names

Now let's print the trait values alongside the corresponding keys.

In [ ]:
traits = {key: getattr(object_has_traits, key) for key in trait_names if object_has_traits.trait_has_value(key)}
traits

Looks pretty good! Let's turn that into a function.

## #| export

Notice the `#| export` statement at the top of cell below. This tells nbdev that we want to export this code to a clean python file. 

In [ ]:
#| export
import pprint

def pprint_traits(has_traits):
    trait_names = object_has_traits.trait_names()
    trait_names = [name for name in trait_names if not name.startswith('_')]
    exclude = ['comm', 'log', 'keys']
    trait_names = [name for name in trait_names if name not in exclude]
    traits = {key: getattr(object_has_traits, key) for key in trait_names if object_has_traits.trait_has_value(key)}
    pprint.pprint(traits)

In [ ]:
pprint_traits(object_has_traits)

Looks good! Let's use `nbdev` to export this function to a python file. 

## #| default_exp
This requires that we add a default export directive to our notebook. It tells nbdev what file to export to.

In [ ]:
#| default_exp helpers

Usually we put this at the top of our notebook, but we can leave it here for now. 

## nb_export

Next we can use the nb_export function to compile the exported cells into a python file. The second parameter tells nbdev that the python file is going in the dashboard directory.

In [ ]:
from nbdev.export import nb_export

nb_export('_nbdev.ipynb', 'dashboard')
nb_export('_nbdev.ipynb', '.dashboard_key')

Given this information, where do you think the code will be exported? Modify the load function with your guess.

In [ ]:
%load dashboard/helpers.py # load path/to/file.py

There are a few thing to notice here. One is that we get a warning about this file being autogenerated. That is, if we make changes to `dashboard/herlpers.py`, those changes will be overwritten the next time we export `../_nbdev.ipynb`.

The second thing to notice is that there is a handy comment to tell us where the code came from. `# %% ../_nbdev.ipynb 13` tells us that the code came from the 13th code cell. 

Another way to verify that this worked is to import the function we just wrote to the python file. Can you guess what that import statement would look like?

In [ ]:
from dashboard.helpers import pprint_traits # import the function we just wrote

Great job! Now we know the basic idea of what nbdev does.

## Literate Programming

Of course, nbdev has a lot of other nice features, but the notebook export is the core of literate programming. You can do your work incrementally, leave in a lot of markdown, and read your code like a book. At the same time, you can keep the code in clean, importable python files. 